In [22]:
import pandas as pd
import requests
import requests_cache
import re
from bs4 import BeautifulSoup as bs

requests_cache.install_cache("cache4")

In [158]:
recipes = pd.read_json('full_format_recipes.json')
r = recipes.dropna(how='all')

no_nutri = recipes[pd.isna(recipes.calories)]

In [157]:
r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20111 entries, 0 to 20129
Data columns (total 11 columns):
calories       15976 non-null float64
categories     20111 non-null object
date           20111 non-null datetime64[ns]
desc           13495 non-null object
directions     20111 non-null object
fat            15908 non-null float64
ingredients    20111 non-null object
protein        15929 non-null float64
rating         20100 non-null float64
sodium         15974 non-null float64
title          20111 non-null object
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 1.8+ MB


In [155]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20130 entries, 0 to 20129
Data columns (total 11 columns):
calories       15976 non-null float64
categories     20111 non-null object
date           20111 non-null datetime64[ns]
desc           13495 non-null object
directions     20111 non-null object
fat            15908 non-null float64
ingredients    20111 non-null object
protein        15929 non-null float64
rating         20100 non-null float64
sodium         15974 non-null float64
title          20111 non-null object
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 1.7+ MB


In [122]:
list(no_nutri.title)[:20]

['Mahi-Mahi in Tomato Olive Sauce ',
 'Ham and Spring Vegetable Salad with Shallot Vinaigrette ',
 'Spicy-Sweet Kumquats ',
 'Spicy Noodle Soup ',
 '"Fried" Chicken ',
 'Fish Fillets in Parchment with Asparagus and Orange ',
 'Pancetta and Taleggio Lasagna with Treviso ',
 'Asian Steak Topped with Bell Pepper Stir-Fry ',
 'Moroccan-Style Preserved Lemons ',
 'Fontina Mac with Squash and Sage ',
 'Pastry Twists with Spiced Sugar-Honey Glaze ',
 'Honey Rye ',
 'Purple-Potato and Crab Gratin ',
 'Grilled Beef, Jícama, and Apple Salad ',
 'Herbed Goat Cheese Spread with Mint ',
 'Braised Chicken With Artichokes and Olives ',
 'Collard-and-Prosciutto Chicken Roulades Over Watercress Salad ',
 'Orange-Almond Cake with Chocolate Icing ',
 'Anadama Rolls with Mixed Seeds ',
 'Bacon Crackers ']

In [154]:
recipes[pd.isna(recipes.title)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 1076 to 19547
Data columns (total 11 columns):
calories       0 non-null float64
categories     0 non-null object
date           0 non-null datetime64[ns]
desc           0 non-null object
directions     0 non-null object
fat            0 non-null float64
ingredients    0 non-null object
protein        0 non-null float64
rating         0 non-null float64
sodium         0 non-null float64
title          0 non-null object
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 1.8+ KB


In [152]:
no_nutri.iloc[213]

calories       NaN
categories     NaN
date           NaT
desc           NaN
directions     NaN
fat            NaN
ingredients    NaN
protein        NaN
rating         NaN
sodium         NaN
title          NaN
Name: 1076, dtype: object

In [33]:
recipes.title[0].strip().replace(' ','%20')

'Lentil,%20Apple,%20and%20Turkey%20Wrap'

In [137]:
import unidecode

In [142]:
recs = [e for e in no_nutri.title if e is not null]
recs_clean = []
# recs_clean = [unidecode.unidecode(rec) for rec in recs]
for rec in recs:
    recs_clean.append(unidecode.unidecode(rec))

AttributeError: 'float' object has no attribute 'encode'

In [151]:
recs.index('Green Chile Charmoula ')
recs[213]

nan

In [143]:
recs_clean

['Mahi-Mahi in Tomato Olive Sauce ',
 'Ham and Spring Vegetable Salad with Shallot Vinaigrette ',
 'Spicy-Sweet Kumquats ',
 'Spicy Noodle Soup ',
 '"Fried" Chicken ',
 'Fish Fillets in Parchment with Asparagus and Orange ',
 'Pancetta and Taleggio Lasagna with Treviso ',
 'Asian Steak Topped with Bell Pepper Stir-Fry ',
 'Moroccan-Style Preserved Lemons ',
 'Fontina Mac with Squash and Sage ',
 'Pastry Twists with Spiced Sugar-Honey Glaze ',
 'Honey Rye ',
 'Purple-Potato and Crab Gratin ',
 'Grilled Beef, Jicama, and Apple Salad ',
 'Herbed Goat Cheese Spread with Mint ',
 'Braised Chicken With Artichokes and Olives ',
 'Collard-and-Prosciutto Chicken Roulades Over Watercress Salad ',
 'Orange-Almond Cake with Chocolate Icing ',
 'Anadama Rolls with Mixed Seeds ',
 'Bacon Crackers ',
 'Ricotta Omelets ',
 'Carrot, Snow Pea, and Red Pepper Julienne in Honey Vinaigrette ',
 'Grilled Steak Salad with Beets and Scallions ',
 'Short Rib Pot Pie ',
 'Sausage Fennel Stuffing ',
 'Baked Pear

In [125]:
base_url = 'https://www.epicurious.com/'
rec_urls = []

def get_url(terms):
    
#     search_urls = [base_url + term.replace(' ', '%20') for term in terms]
    for term in terms:
        page = 1
        while page < 11:
            search_url = f"{base_url}search/{term.strip().replace(' ', '%20')}?content=recipe&page={page}"
            regex = f"^/recipes/food/views/{term.strip().replace(' ', '-').lower()}-[0-9]{{1,6}}"
            regex = regex.replace('"', '')
#         print(search_url, regex)
            req = requests.get(search_url)
            soup = bs(req.text, 'lxml')
        
            try:
                rec_urls.append(base_url + soup.find('a', attrs={'href': re.compile(regex)})['href'])
                break
            except TypeError:
                page += 1
            if page == 10:
                print('Unsuccessful')
        print(f"{rec_urls[-1]}: {len(rec_urls) - 1}")
                
        
#         search_url = base_url + 'search/' + term.strip().replace(' ', '%20')
#         regex = '^/recipes/food/views/' + term.strip().replace(' ', '-').lower() + '-' + '[0-9]{1,6}'
# #         print(search_url, regex)
#         req = requests.get(search_url)
#         soup = bs(req.text, 'lxml')
#         try:
#             rec_urls.append(base_url + soup.find('a', attrs={'href': re.compile(regex)})['href'])
#         except TypeError:
#             for page in range(2,10):
#                 search_url += '?page=' + page
#                 req = 
#     return rec_urls
        
#     for url in urls:
#         req = requests.get(url)
#         soup = bs(req.text, 'lxml')
        
    
    
#     return urls

        


In [126]:
# get_url(list(no_nutri.title)[0])
# get_url([no_nutri.title.iloc[0]])
get_url(list(no_nutri.title[:20]))

https://www.epicurious.com//recipes/food/views/mahi-mahi-in-tomato-olive-sauce-352551: 0
https://www.epicurious.com//recipes/food/views/ham-and-spring-vegetable-salad-with-shallot-vinaigrette-5212: 1
https://www.epicurious.com//recipes/food/views/spicy-sweet-kumquats-102726: 2
https://www.epicurious.com//recipes/food/views/spicy-noodle-soup-103188: 3
https://www.epicurious.com//recipes/food/views/fried-chicken-105048: 4
https://www.epicurious.com//recipes/food/views/fish-fillets-in-parchment-with-asparagus-and-orange-241849: 5
https://www.epicurious.com//recipes/food/views/pancetta-and-taleggio-lasagna-with-treviso-356354: 6
https://www.epicurious.com//recipes/food/views/asian-steak-topped-with-bell-pepper-stir-fry-350398: 7
https://www.epicurious.com//recipes/food/views/moroccan-style-preserved-lemons-242341: 8
https://www.epicurious.com//recipes/food/views/fontina-mac-with-squash-and-sage-51206440: 9
https://www.epicurious.com//recipes/food/views/pastry-twists-with-spiced-sugar-honey

KeyboardInterrupt: 

In [78]:
print(f'this is {base_url}')

this is https://www.epicurious.com/


In [67]:
c = 0
l=['this', 'that','1','2']
while True:
    try:
        int(l[c])
        print('success' + l[c])
        break
    except: 
        print('cant convert ' + l[c] + 'to int')
        c += 1

cant convert thisto int
cant convert thatto int
success1


In [127]:
rec_urls

['https://www.epicurious.com//recipes/food/views/mahi-mahi-in-tomato-olive-sauce-352551',
 'https://www.epicurious.com//recipes/food/views/ham-and-spring-vegetable-salad-with-shallot-vinaigrette-5212',
 'https://www.epicurious.com//recipes/food/views/spicy-sweet-kumquats-102726',
 'https://www.epicurious.com//recipes/food/views/spicy-noodle-soup-103188',
 'https://www.epicurious.com//recipes/food/views/fried-chicken-105048',
 'https://www.epicurious.com//recipes/food/views/fish-fillets-in-parchment-with-asparagus-and-orange-241849',
 'https://www.epicurious.com//recipes/food/views/pancetta-and-taleggio-lasagna-with-treviso-356354',
 'https://www.epicurious.com//recipes/food/views/asian-steak-topped-with-bell-pepper-stir-fry-350398',
 'https://www.epicurious.com//recipes/food/views/moroccan-style-preserved-lemons-242341',
 'https://www.epicurious.com//recipes/food/views/fontina-mac-with-squash-and-sage-51206440',
 'https://www.epicurious.com//recipes/food/views/pastry-twists-with-spiced

In [44]:
re.compile('^/recipes/food/views/Mahi-Mahi-in-Tomato-Olive-Sauce-[0-9]{6}')

re.compile(r'^/recipes/food/views/Mahi-Mahi-in-Tomato-Olive-Sauce-[0-9]{6}',
re.UNICODE)

In [35]:
no_nutri.title.iloc[0]

'Mahi-Mahi in Tomato Olive Sauce '

In [17]:
help(str.replace)

Help on method_descriptor:

replace(...)
    S.replace(old, new[, count]) -> str
    
    Return a copy of S with all occurrences of substring
    old replaced by new.  If the optional argument count is
    given, only the first count occurrences are replaced.

